# 其他

In [1]:
from timevar import savetime
import time 
import random
from time import sleep
from bs4 import BeautifulSoup

In [2]:
def downpmc(limit):
    tablename = 'pubmed%s'%savetime
    count=0
    dbpath = 'pubmedsql'
    result = readdata1(dbpath, 1)
    for item in result:
        count+=1

        if count > limit:
            print("已达到需要下载的上限，下载停止\n")
            break
        print("开始下载第%d篇"%count)
        #result是从数据库获取的列表元组，其中的每一项构成为PMCID,doctitle
        html=downpdf(item[0])
        if html==None:
            print("网页pdf数据不存在，自动跳过该文献 PMCID为 %s" % item[0])
            continue
        elif html == 1:
            print("30s超时,自动跳过该文献 PMCID为 %s" % item[0])
            continue
        status = savepdf(html, item[0], item[1], dbpath)
        if status == None:
            print("保存pdf文件发生错误，自动跳过该文献PMCID为 %s" % item[0])
            continue
        time.sleep(random.randint(0, 1))
    save2excel(dbpath)  # 这里我把默认的数据库路径改成了全局变量dbpath
    print("爬取最终结果信息已经自动保存到excel表格中，文件名为%s" % tablename)
    print("爬取的所有文献已经保存到/document/pub/目录下")
    print("爬取程序已经执行完成，自动退出, 哈哈，no errors no warning")


pubmed20240605124322
SELECT PMCID,doctitle FROM pubmed20240605124322 WHERE freemark = '2'
目标数据库读取失败



TypeError: 'NoneType' object is not iterable

In [2]:
# -*- coding: utf-8 -*-
import random
import re
import sqlite3
import time
import urllib
import urllib.error
from urllib import request

import eventlet
import xlwt

from geteachinfo import readdata1
from timevar import savetime


In [4]:
def downpdf(parameter):
    eventlet.monkey_patch()
    downpara = "pmc/articles/" + parameter + "/pdf/main.pdf"
    # openurl是用于使用指定的搜索parameter进行检索，以get的方式获取pubmed的搜索结果页面，返回成html文件
    baseurl = "https://www.ncbi.nlm.nih.gov/"
    url = baseurl + downpara
    print(f"\033[1;35m[I] download url: {url}\033[0m")
    timeout_flag=0
    header={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36 Edg/101.0.1210.32"}
    request=urllib.request.Request(url,headers=header)
    html=""
    # try:
    #     response=urllib.request.urlopen(request,timeout=30)
    #     html=response.read()
    #     print("%s.pdf"%parameter,"从目标站获取pdf数据成功")
    #     return html
    try:
        with eventlet.Timeout(180,True):
            response = urllib.request.urlopen(request, timeout=60)
            html = response.read()
            print("%s.pdf" % parameter, "从目标站获取pdf数据成功")
            return html
    except urllib.error.URLError as e:
        if hasattr(e, "code"):  # 判断e这个对象里面是否包含code这个属性
            print(e.code)
        if hasattr(e, "reason"):
            print(e.reason)
        timeout_flag=1
        return timeout_flag
    except eventlet.timeout.Timeout:
        timeout_flag=1
        print("下载目标数据超时")
        return timeout_flag
    except:
        print("%s.pdf"%parameter,"从目标站获取pdf数据失败")


In [4]:
url = 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8835688/pdf/main.pdf'
header={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36 Edg/101.0.1210.32"}
request=urllib.request.Request(url,headers=header)
html=""
response = urllib.request.urlopen(request, timeout=60)
html = response.read()


In [6]:
file = open('test.pdf', 'wb')
file.write(html)
file.close()

In [ ]:
def save_pdf_from_url(url, saved_filename):
    header={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36 Edg/101.0.1210.32"}
    request=urllib.request.Request(url,headers=header)
    html=""
    response = urllib.request.urlopen(request, timeout=60)
    html = response.read()
    file = open(saved_filename, 'wb')
    file.write(html)
    file.close()

In [10]:
import hashlib
hashlib.md5(html).hexdigest()

'edce97fdb7759f09a77316e42f5bef8f'

In [6]:
class Search_param(str):
    """
    可以由generate方法encode生成初次检索所需的URL后缀
    例如：”?term=alzheimer%27s+disease“
    可以附加一些调整网页的属性
    """
    def __init__(self, keywords:str):
        self.search_keywords = {}
        self.search_keywords['term'] = keywords.strip()

    def gen_search_param(self) -> str:
        # encode url生成request需要的url
        return urllib.parse.urlencode(self.search_keywords)

    def specify_web_size(self, size: int):
        # 调整 搜索页面的大小
        self.search_keywords['size'] = size

    def specify_any_param(self, key: str, value):
        """
        针对任意参数进行调整, 需要提供合适的键值对，默认不存在
        目前观察到的有：sort(date, pubdate, fauth, jour), sort_order(asc) 更多参数请查看pubmed的搜索URL
        :param key: url链接需要添加的键
        :param value: url链接中键对应的值
        :return:
        """
        self.search_keywords[key] = value

In [7]:
def opensearch(parameter):
    """ openurl是用于使用指定的搜索 parameter 进行检索，以get的方式获取 pubmed 的搜索结果页面，返回成 html 文件 """
    baseurl = "https://pubmed.ncbi.nlm.nih.gov/"
    url=baseurl+"?"+parameter
    print(f'url is {url}')
    header={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36 Edg/101.0.1210.32"}
    request=urllib.request.Request(url,headers=header)
    html=""
    try:
        response=urllib.request.urlopen(request)
        html=response.read()
        return html
    # except urllib.error.URLError as e:
    #     if hasattr(e,"code"):
    #         print(e.code)
    #     if hasattr(e,"reason"):
    #         print(e.reason)
    except:
        print("获取检索页失败，请检查输入的参数\n")

In [8]:
def traverse(html) -> list:
    global result_all
    findlink_title = re.compile(r'<a class="docsum-title".*?">.*?([A-Z0-9].*?)[.?]', re.S)  # 提取文章的title，含有</b>符号。每页有50个文章的title
    findlink_PMID = re.compile(r'<span class="citation-part".*>(\d+)<\/span>')  # 提取PMID号，作为下载的地址
    findlink_FREE_PMC_MARK = re.compile(r'<span class="free-resources.*?>(.*)\.<')  # 提取free pmc article标志
    # findlink4 = re.compile(r'<span class="value">(.+)<\/span>')
    # findlink5 = re.compile(r'(doi:.*?\.).?</span>')
    # findlink5 = re.compile(r'(doi:.*?\.)[ <]')#查找文献doi
    findlink_journal = re.compile(r'journal-citation">(.*?) doi:.*?\.[ <]')  # 查找期刊名称
    findlink_doi = re.compile(r'journal-citation">.*? (doi:.*?\.)[ <]')  # 查找文献doi
    findlink_auto_info = re.compile(r'full-authors">(.*?)<\/span>')  # 查找作者名称信息
    findlink_review = re.compile(r'citation-part">Review.<\/span>')  # 查找文献review标签

    try:
        content = BeautifulSoup(html, "html.parser")  # 通过bs4解析网页的内容，查找到页面有关于各个项目的信息
        result_all = int(str(content.find_all("span", class_="value")[0])[20:-7].replace(',', '')) 
        # 返回 <span class="value">498,730</span> 中的 498730。网页展示 498,730 results

        # page_now = re.match(findlink4,str(content))
        # print(page_now)
        data = []
        for item in content.find_all("div", class_="docsum-content"):
            item = str(item)
            temp = []

            PMID = re.findall(findlink_PMID, item)[0]

            journal = re.search(findlink_journal, item)  # 少数文献是没有doi号的，直接用item[0]会导致index超出
            if journal == None:
                journal = ''
            else:
                journal = journal.group(1)
            print(journal)

            doi = re.search(findlink_doi, item)  # 少数文献是没有doi号的，直接用item[0]会导致index超出
            if doi == None:
                doi = ''
            else:
                doi = doi.group(1)
            print("文献doi号:%s" % doi)
            # 下面是获取作者名单#
            authorlist = re.search(findlink_auto_info, item)
            if authorlist != None:
                authorlist = authorlist.group(1)
            else:
                authorlist = ''

            # 下面是查找文献的title
            doctitle = re.findall(findlink_title, item)[0]
            # doctitle=doctitle.replace("<b>",'').replace("</b>",'').replace("<em>",'').replace("</em>",'').replace('<sup>','').replace("</sup>",'').replace("-/-",'')
            doctitle = doctitle.replace("-/-", '')
            doctitle = re.sub(r"<.+?>", '', doctitle)

            # 查找文献的期刊名称和分布的时间

            # 下面是freemark，分为两种类型，free pmc article和free article，没有的为空值
            # freemark flag: 0，不是免费文件无原文，1 是free article无原文， 2 是有pmc原文
            freemark = re.findall(findlink_FREE_PMC_MARK, item)
            # print(freemark)
            # print(len(freemark))
            if len(freemark) == 0:
                freemark = '0'
            else:
                if len(freemark[0]) == 16:
                    freemark = '2'
                else:
                    freemark = '1'
            # doctitle=doctitle.replace("</b>",'')

            # 下面是查找review标签，两种，空值或者有，即0或者1，1就是表示文章是review类型的
            reviewmark = re.search(findlink_review, item)
            # print(reviewmark.group())
            if reviewmark == None:
                reviewmark = '0'
            else:
                reviewmark = '1'
            temp.append(doctitle)
            temp.append(authorlist)
            temp.append(journal)
            temp.append(doi)
            temp.append(PMID)
            temp.append(freemark)
            temp.append(reviewmark)
            data.append(temp)
            print("当前data数据长度%d" % len(data))
            print('\n')
        return data
    except Exception as e:
        print(f"遍历搜索页信息失败, {e}\n")


In [12]:

def spiderpub(parameter: str, num1: int):
    size = re.search('&size=(\d{1,3})', parameter)
    # if size == None:
    #     parameter += '&size=50'
    #     parameter = re.sub(r'&size=(\d{1,3})', '&size=50', parameter)
    #     print(f'parameter is {parameter} with re.sub ')
    # print(f'parameter is {parameter} without re.sub ')
    page_count = 0
    datalist = []
    result_all_num = -1
    pagemax = 1
    for i in range(1, num1 + 1):
        # 开始遍历每一页结果，一共num页最大pagemax页
        if i > pagemax:
            print("已遍历所有页\n")
            break
        parameter = parameter + "&page=" + str(i)
        try:
            html = opensearch(parameter)
            if html == None:
                print("检索页获取出错，即将退出\n")
                break
            data = traverse(html)
            if data==None:
                print("遍历检索页信息出错，当前检索页为%d(每页50个结果)\n"%i)
            datalist.extend(data)
            pagemax = (result_all + 49) // 50
            sleep(random.randint(0, 2))
            print("已爬取完第%d页\n" % i)
            # print(datalist)
        except:
            print("spiderpub函数出错，请检查结果\n")
  

    txtname="pubmed%s.txt"%savetime
    file = open(txtname, "w", encoding='utf-8')
    for item in datalist:
        output = ''
        for j in item:
            output = output + str(j) + ' '
        output = output + '\n'
        file.write((output))
    file.close
    return datalist


In [11]:
search_param = Search_param("neurosurgery")
search_param.specify_web_size(3)
page = 2 # 要爬的总页数
spiderpub(search_param.gen_search_param(), page)


parameter is term=neurosurgery&size=3 before re.sub 
parameter is term=neurosurgery&size=50 after re.sub 
url is https://pubmed.ncbi.nlm.nih.gov/?term=neurosurgery&size=50&page=1
当前关键词共有498730个搜索结果
Int J Environ Res Public Health. 2022 Feb 2;19(3):1719.
文献doi号:doi: 10.3390/ijerph19031719.
当前data数据长度1


Neurosurg Clin N Am. 2022 Apr;33(2):233-239.
文献doi号:doi: 10.1016/j.nec.2021.11.007.
当前data数据长度2


World Neurosurg. 2022 Nov;167:28-36.
文献doi号:doi: 10.1016/j.wneu.2022.08.067.
当前data数据长度3


World Neurosurg. 2018 May;113:411-424.
文献doi号:doi: 10.1016/j.wneu.2018.01.086.
当前data数据长度4


World Neurosurg. 2018 May;113:436-452.
文献doi号:doi: 10.1016/j.wneu.2018.01.085.
当前data数据长度5


World Neurosurg. 2017 Aug;104:39-44.
文献doi号:doi: 10.1016/j.wneu.2017.04.150.
当前data数据长度6


J Clin Neurosci. 2021 Apr;86:347-356.
文献doi号:doi: 10.1016/j.jocn.2021.01.054.
当前data数据长度7


Neurosurgery. 2022 Oct 1;91(4):527-528.
文献doi号:doi: 10.1227/neu.0000000000002133.
当前data数据长度8


World Neurosurg. 2022 Oct;166:184-190.
文献d

[['Virtual Reality in Neurosurgery: Beyond Neurosurgical Planning',
  'Mishra R, Narayanan MDK, Umana GE, Montemurro N, Chaurasia B, Deora H.',
  'Int J Environ Res Public Health. 2022 Feb 2;19(3):1719.',
  'doi: 10.3390/ijerph19031719.',
  '35162742',
  '2',
  '1'],
 ['Future Directions of Endovascular Neurosurgery',
  'Yaeger K, Mocco J.',
  'Neurosurg Clin N Am. 2022 Apr;33(2):233-239.',
  'doi: 10.1016/j.nec.2021.11.007.',
  '35346456',
  '0',
  '1'],
 ['The Current State of Neurosurgery in Iceland',
  'Róbertsson V, Shlobin NA, Magnadóttir H, Guðmundsson KR, Ólafsson IH, Björnsson A, Rosseau G.',
  'World Neurosurg. 2022 Nov;167:28-36.',
  'doi: 10.1016/j.wneu.2022.08.067.',
  '36028110',
  '0',
  '1'],
 ['Neurosurgery in East Africa: Foundations',
  'Mangat HS, Schöller K, Budohoski KP, Ngerageza JG, Qureshi M, Santos MM, Shabani HK, Zubkov MR, Härtl R, Stieg PE.',
  'World Neurosurg. 2018 May;113:411-424.',
  'doi: 10.1016/j.wneu.2018.01.086.',
  '29702965',
  '0',
  '1'],
 ['Ne

In [ ]:
https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=Operative+Landscape+at+Canadian+Neurosurgery+Residency+Programs&btnG=

https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=Operative+Landscape+at+Canadian+Neurosurgery+Residency+Programs&btnG=

# 函数块

In [5]:
from tqdm import tqdm

from timevar import savetime
import time 
import random
from time import sleep
from bs4 import BeautifulSoup

# -*- coding: utf-8 -*-
import random
import re
import sqlite3
import time
import urllib
import urllib.error
from urllib import request

import eventlet
import xlwt

from geteachinfo import readdata1
from timevar import savetime


In [68]:

class Search_param(str):
    """
    搜索参数管理类，用于拼接在baseurl之后

    可以由generate方法encode生成初次检索所需的URL后缀
    例如：”?term=alzheimer%27s+disease“
    可以附加一些调整网页的属性

    TODO: 按日期搜索，
    2020.1.1 - 2020.1.31: 
    https://pubmed.ncbi.nlm.nih.gov/?term=computer+science&filter=dates.2020%2F1%2F1-2020%2F1%2F31

    2024.5.1 - 2024.5.31
    https://pubmed.ncbi.nlm.nih.gov/?term=computer+science&filter=dates.2024%2F5%2F1-2024%2F5%2F31
    """
    def __init__(self, keywords:str):
        self.search_keywords = {}
        self.search_keywords['term'] = keywords.strip()

    def gen_search_param(self) -> str:
        # encode url生成request需要的url
        return urllib.parse.urlencode(self.search_keywords)

    def specify_web_size(self, size: int):
        # 调整 搜索页面的大小
        self.search_keywords['size'] = size

    def specify_any_param(self, key: str, value):
        """
        针对任意参数进行调整, 需要提供合适的键值对，默认不存在
        目前观察到的有：sort(date, pubdate, fauth, jour), sort_order(asc) 更多参数请查看pubmed的搜索URL
        :param key: url链接需要添加的键
        :param value: url链接中键对应的值
        :return:
        """
        self.search_keywords[key] = value

def spider_pubmed_paper_metadata(parameter: str = "computer science", startpage: int = 1,  endpage: int = 12):
    """ return: metadata of papers
        爬虫函数主体 
        startpage 是从搜索结果第几页起爬
        endpage 是搜索结果第几页结束爬
        返回： datalist: list = [] 每个元素是一篇论文的情况
        每个元素的内容： 
        ```
        0: document title
        1: author list
        2: journal 
        3: doi 
        4: PMID
        5: freemark, 是否free, 2 是有原文的
        6: reviewmark, 是否 review
        ```
    """
    datalist = [] # startpage 到 endpage 的页中的论文列表


    baseurl = "https://pubmed.ncbi.nlm.nih.gov/"
    url=""
    header={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36 Edg/101.0.1210.32"}

    # 先获取所有搜索结果：
    result_all = -1

    # 下列正则表达式用来搜索
    findlink_PMID = re.compile(r'<span class="citation-part".*>(\d+)<\/span>')  # 提取PMID号，作为下载的地址
    findlink_journal = re.compile(r'journal-citation">(.*?) doi:.*?\.[ <]')  # 查找期刊名称
    findlink_doi = re.compile(r'journal-citation">.*? (doi:.*?\.)[ <]')  # 查找文献doi
    findlink_title = re.compile(r'<a class="docsum-title".*?">.*?([A-Z0-9].*?)[.?]', re.S) # 提取文章的title，含有</b>符号。每页有50个文章的title
    findlink_auto_info = re.compile(r'full-authors">(.*?)<\/span>')  # 查找作者名称信息
    findlink_FREE_PMC_MARK = re.compile(r'<span class="free-resources.*?>(.*)\.<')  # 提取free pmc article标志
    findlink_review = re.compile(r'citation-part">Review.<\/span>')  # 查找文献review标签

    progress_bar = tqdm(total=endpage - startpage)
    for i in range(startpage, endpage):

        # [1] 开始遍历每一页结果，一共 page 页最大pagemax页
        

        # [2] 用指定的参数， get 的方式获取 pubmed 的搜索结果 
        url=baseurl+"?"+ parameter + "&page=" + str(i)
        request=urllib.request.Request(url,headers=header)
        response=urllib.request.urlopen(request)
        # print(f'\033[1;35m url is {url}\033[0m')
        html=response.read()

        content = BeautifulSoup(html, "html.parser")
        # [3] 计算 result_all 是 <span class="value">498,730</span> 中的 498730。网页展示 498,730 results
        result_all = int(str(content.find_all("span", class_="value")[0])[20:-7].replace(',', '')) 
        

        # [3] 把一个页面的 50 条结果全部返回成一个 list
        #data = traverse(html)
        data = []
        # print(f'\033[1;35m len of item is {len(content.find_all("div", class_="docsum-content"))}\033[0m')
        for item in content.find_all("div", class_="docsum-content"):
            ''' 获取 PMID, 作者, 期刊, doi, title, 是否free, 是否 review '''
            item = str(item)
            
            # 获取 PMID
            PMID = re.findall(findlink_PMID, item)[0]

            # 获取作者
            authorlist = re.search(findlink_auto_info, item)
            if authorlist != None:
                authorlist = authorlist.group(1)
            else:
                authorlist = ''

            # 获取期刊
            journal = re.search(findlink_journal, item)  
            if journal == None:
                journal = ''
            else:
                journal = journal.group(1)


            # 获取 doi
            doi = re.search(findlink_doi, item)  # 少数文献是没有doi号的，直接用item[0]会导致index超出
            if doi == None:
                doi = ''
            else:
                doi = doi.group(1)

            # 获取 title
            doctitle = re.findall(findlink_title, item)[0]
            doctitle = doctitle.replace("-/-", '')
            doctitle = re.sub(r"<.+?>", '', doctitle)

            # freemark flag: 0，不是免费文件无原文，1 是free article无原文， 2 是有pmc原文
            freemark = re.findall(findlink_FREE_PMC_MARK, item)
            if len(freemark) == 0:
                freemark = '0'
            else:
                if len(freemark[0]) == 16:
                    freemark = '2'
                else:
                    freemark = '1'

            # 1 表示文章类型是 review
            reviewmark = re.search(findlink_review, item)
            # print(reviewmark.group())
            if reviewmark == None:
                reviewmark = '0'
            else:
                reviewmark = '1'

            temp = []
            temp.append(doctitle)
            temp.append(authorlist)
            temp.append(journal)
            temp.append(doi)
            temp.append(PMID)
            temp.append(freemark)
            temp.append(reviewmark)
            data.append(temp)

        progress_bar.update(1)  # 更新进度条
        datalist.extend(data) # 直接增加到datalist里， 每个元素就是一篇论文的情况
        sleep(random.randint(0, 2))
    progress_bar.close()
    return datalist

def download_pubmed_pdf_using_paper_metadata(metadata):
    # metadata 中没有 PMCID，只有PMID。下面获取PMCID
    baseurl = "https://pubmed.ncbi.nlm.nih.gov/"  # baseurl和之前的搜索页面一致
    PMID = str(metadata[4])
    url = baseurl + PMID
    header = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36 Edg/101.0.1210.32"}
    request = urllib.request.Request(url, headers=header)
    html = ""
    response = urllib.request.urlopen(request)
    html = response.read()
    content = BeautifulSoup(html, "html.parser")
    heading = content.find_all('div', class_="full-view", id="full-view-heading")
    heading = str(heading)  # heading是一个包含文章大部分信息的标签
    PMCID = re.search(r'(PMC\d+)\n', heading)  # 获取PMCID用于后续的自动下载
    PMCID = PMCID.group(1)
    # 这一页包括了PMCID和全文等其他部分。但在这里，只需要PMCID就能下载到PDF

    # 尝试直接从 pubmed 下载

    '''
    
    https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8022506/pdf/peerj-cs-07-441.pdf
    https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8022506/pdf

    29702967
    https://www.ncbi.nlm.nih.gov/pmc/articles/PMC29702967/
    https://www.ncbi.nlm.nih.gov/pmc/articles/PMC36091982/pdf

    https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7319935/pdf/main.pdf 打得开, PMID 是 32599201

    PMID 36091982， 题目，Automatic computer science domain multiple-choice questions generation based on informative sentences
    PMCID：
    '''

    downpara = "pmc/articles/" + PMCID + "/pdf"
    baseurl = "https://www.ncbi.nlm.nih.gov/"
    url = baseurl + downpara
    print(f"\033[1;35m[I] pdf url: {url}\033[0m", end = ".")
    header={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36 Edg/101.0.1210.32"}
    request=urllib.request.Request(url,headers=header)
    html=""
    response = urllib.request.urlopen(request, timeout=60)
    html = response.read()
    # print("%s.pdf" % metadata[0], "从目标站获取pdf数据成功")
    return html

def fetch_pdf_urls_from_pubmed(keyword = 'computer science', startpage = 1, endpage = 20):
    search_param = Search_param(keyword)
    metadatas = spider_pubmed_paper_metadata(search_param.gen_search_param(), startpage = startpage, endpage = endpage)
    return metadatas



In [54]:
metadatas = fetch_pdf_urls_from_pubmed(keyword = 'neurosurgery', startpage = 1, endpage = 10)

100%|██████████| 9/9 [00:22<00:00,  2.55s/it]


In [ ]:
for metadata in metadatas:
    try:
        pdfstream = download_pubmed_pdf_using_paper_metadata(metadata)
        savepath = f"./document/neurosurgery/{metadata[4]}.pdf"  # 这里用 PMID 存文件
        file = open(savepath, 'wb')
        file.write(pdfstream)
        file.close()
    except Exception as e:
        print(e, f"PMID: {metadata[4]}")

In [ ]:
metadatas = fetch_pdf_urls_from_pubmed(keyword = 'neurosurgery', startpage = 11, endpage = 100)
import logging
from datetime import datetime

# 配置日志
logging.basicConfig(filename='error.log', level=logging.ERROR,
                    format='%(asctime)s:%(levelname)s:%(message)s')


In [69]:
length = len(metadatas)
i = 0
for metadata in metadatas:
    print(f'{i}/{length}', end = ',')
    try:
        pdfstream = download_pubmed_pdf_using_paper_metadata(metadata)
        savepath = f"./document/neurosurgery/{metadata[4]}.pdf"  # 这里用 PMID 存文件
        file = open(savepath, 'wb')
        file.write(pdfstream)
        file.close()
        print('success', end = '\n')
    except Exception as e:
        # 将异常信息写入日志文件
        logging.error(f"Error: {e}, PMID: {metadata[4]}")
        print('\n')
    i = i + 1

0/890,

1/890,

2/890,

3/890,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2293965/pdf.success
4/890,

5/890,

6/890,

7/890,

8/890,

9/890,

10/890,

11/890,

12/890,

13/890,

14/890,

15/890,

16/890,

17/890,

18/890,

19/890,

20/890,

21/890,

22/890,

23/890,

24/890,

25/890,

26/890,

27/890,

28/890,

29/890,

30/890,

31/890,

32/890,

33/890,

34/890,

35/890,

36/890,

37/890,

38/890,

39/890,

40/890,

41/890,

42/890,

43/890,

44/890,

45/890,

46/890,

47/890,

48/890,

49/890,

50/890,

51/890,

52/890,

53/890,

54/890,

55/890,

56/890,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7603429/pdf.success
57/890,

58/890,

59/890,

60/890,

61/890,

62/890,

63/890,

64/890,

65/890,

66/890,

67/890,

68/890,

69/890,

70/890,

71/890,

72/890,

73/890,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9212868/pdf.success
74/890,

75/890,

76/890,

77/890,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC11019977/pdf.success
78/

In [70]:
metadatas = fetch_pdf_urls_from_pubmed(keyword = 'neurosurgery', startpage = 101, endpage = 200)
import logging
from datetime import datetime

# 配置日志
logging.basicConfig(filename='error.log', level=logging.ERROR,
                    format='%(asctime)s:%(levelname)s:%(message)s')


100%|██████████| 99/99 [05:56<00:00,  3.61s/it]


In [75]:
# 配置日志
logging.basicConfig(filename='error.log', level=logging.ERROR,
                    format='%(asctime)s:%(levelname)s:%(message)s')
logging.error(f"none")

In [71]:
length = len(metadatas)
i = 0
for metadata in metadatas:
    print(f'{i}/{length}', end = ',')
    try:
        pdfstream = download_pubmed_pdf_using_paper_metadata(metadata)
        savepath = f"./document/neurosurgery/{metadata[4]}.pdf"  # 这里用 PMID 存文件
        file = open(savepath, 'wb')
        file.write(pdfstream)
        file.close()
        print('success')
    except Exception as e:
        # 将异常信息写入日志文件
        logging.error(f"Error: {e}, PMID: {metadata[4]}")
    i = i + 1

0/990,1/990,2/990,3/990,4/990,5/990,6/990,7/990,8/990,9/990,10/990,11/990,12/990,13/990,14/990,15/990,16/990,17/990,18/990,19/990,20/990,21/990,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5446044/pdf.success
22/990,23/990,24/990,25/990,26/990,27/990,28/990,29/990,30/990,31/990,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9451940/pdf.success
32/990,33/990,34/990,35/990,36/990,37/990,38/990,39/990,40/990,41/990,42/990,43/990,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10166603/pdf.success
44/990,45/990,46/990,47/990,48/990,49/990,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1871628/pdf.success
50/990,51/990,52/990,53/990,54/990,55/990,56/990,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7607232/pdf.success
57/990,58/990,59/990,60/990,61/990,62/990,63/990,64/990,65/990,66/990,67/990,68/990,69/990,70/990,71/990,72/990,73/990,74/990,75/990,76/990,77/990,78/990,79/990,80/990,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articl

KeyboardInterrupt: 

In [77]:
length = len(metadatas)
i = 528
while True:
    print(f'{i}/{length}', end = ',')
    metadata = metadatas[i]
    try:
        pdfstream = download_pubmed_pdf_using_paper_metadata(metadata)
        savepath = f"./document/neurosurgery/{metadata[4]}.pdf"  # 这里用 PMID 存文件
        file = open(savepath, 'wb')
        file.write(pdfstream)
        file.close()
        print('success')
    except Exception as e:
        # 将异常信息写入日志文件
        logging.error(f"Error: {e}, PMID: {metadata[4]}")
    i = i + 1
    if i == length + 1:
        break

528/990,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7499752/pdf.success
529/990,530/990,531/990,532/990,533/990,534/990,535/990,536/990,537/990,538/990,539/990,540/990,541/990,542/990,543/990,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1973231/pdf.success
544/990,545/990,546/990,547/990,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2899738/pdf.success
548/990,549/990,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5480983/pdf.success
550/990,551/990,552/990,553/990,554/990,555/990,556/990,557/990,558/990,559/990,560/990,561/990,562/990,563/990,564/990,565/990,566/990,567/990,568/990,569/990,570/990,571/990,572/990,573/990,574/990,575/990,576/990,577/990,578/990,579/990,580/990,581/990,582/990,583/990,584/990,585/990,586/990,587/990,588/990,589/990,590/990,591/990,592/990,593/990,594/990,595/990,596/990,597/990,598/990,599/990,600/990,601/990,602/990,603/990,604/990,605/990,606/990,607/990,608/990,609/990,610/990,611/990,612/990,613/990,

IndexError: list index out of range

In [78]:
metadatas = fetch_pdf_urls_from_pubmed(keyword = 'neurosurgery', startpage = 200, endpage = 400)
import logging
from datetime import datetime

# 配置日志
logging.basicConfig(filename='error.log', level=logging.ERROR,
                    format='%(asctime)s:%(levelname)s:%(message)s')


100%|██████████| 200/200 [10:09<00:00,  3.05s/it]


In [79]:
length = len(metadatas)
i = 0
while True:
    print(f'{i}/{length}', end = ',')
    metadata = metadatas[i]
    try:
        pdfstream = download_pubmed_pdf_using_paper_metadata(metadata)
        savepath = f"./document/neurosurgery/{metadata[4]}.pdf"  # 这里用 PMID 存文件
        file = open(savepath, 'wb')
        file.write(pdfstream)
        file.close()
        print('success')
    except Exception as e:
        # 将异常信息写入日志文件
        logging.error(f"Error: {e}, PMID: {metadata[4]}")
    i = i + 1
    if i == length + 1:
        break

0/2000,1/2000,2/2000,3/2000,4/2000,5/2000,6/2000,7/2000,8/2000,9/2000,10/2000,11/2000,12/2000,13/2000,14/2000,15/2000,16/2000,17/2000,18/2000,19/2000,20/2000,21/2000,22/2000,23/2000,24/2000,25/2000,26/2000,27/2000,28/2000,29/2000,30/2000,31/2000,32/2000,33/2000,34/2000,35/2000,36/2000,37/2000,38/2000,39/2000,40/2000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8338820/pdf.success
41/2000,42/2000,43/2000,44/2000,45/2000,46/2000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7826024/pdf.success
47/2000,48/2000,49/2000,50/2000,51/2000,52/2000,53/2000,54/2000,55/2000,56/2000,57/2000,58/2000,59/2000,60/2000,61/2000,62/2000,63/2000,64/2000,65/2000,66/2000,67/2000,68/2000,69/2000,70/2000,71/2000,72/2000,73/2000,74/2000,75/2000,76/2000,77/2000,78/2000,79/2000,80/2000,81/2000,82/2000,83/2000,84/2000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7895493/pdf.success
85/2000,86/2000,87/2000,88/2000,89/2000,90/2000,91/2000,92/2000,93/2000,94/2000,95/2000,96/2000,

KeyboardInterrupt: 

In [81]:
len(metadatas)

2000

In [82]:
length = len(metadatas)
i = 703
while True:
    print(f'{i}/{length}', end = ',')
    metadata = metadatas[i]
    try:
        pdfstream = download_pubmed_pdf_using_paper_metadata(metadata)
        savepath = f"./document/neurosurgery/{metadata[4]}.pdf"  # 这里用 PMID 存文件
        file = open(savepath, 'wb')
        file.write(pdfstream)
        file.close()
        print('success')
    except Exception as e:
        # 将异常信息写入日志文件
        logging.error(f"Error: {e}, PMID: {metadata[4]}")
    i = i + 1
    if i == length + 1:
        break

703/2000,704/2000,705/2000,706/2000,707/2000,708/2000,709/2000,710/2000,711/2000,712/2000,713/2000,714/2000,715/2000,716/2000,717/2000,718/2000,719/2000,720/2000,721/2000,722/2000,723/2000,724/2000,725/2000,726/2000,727/2000,728/2000,729/2000,730/2000,731/2000,732/2000,733/2000,734/2000,735/2000,736/2000,737/2000,738/2000,739/2000,740/2000,741/2000,742/2000,743/2000,744/2000,745/2000,746/2000,747/2000,748/2000,749/2000,750/2000,751/2000,752/2000,753/2000,754/2000,755/2000,756/2000,757/2000,758/2000,759/2000,760/2000,761/2000,762/2000,763/2000,764/2000,765/2000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3883460/pdf.success
766/2000,767/2000,768/2000,769/2000,770/2000,771/2000,772/2000,773/2000,774/2000,775/2000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8078872/pdf.success
776/2000,777/2000,778/2000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9084592/pdf.success
779/2000,780/2000,781/2000,782/2000,783/2000,784/2000,785/2000,786/2000,787/2000,7

KeyboardInterrupt: 

In [83]:
length = len(metadatas)
i = 1261
while True:
    print(f'{i}/{length}', end = ',')
    metadata = metadatas[i]
    try:
        pdfstream = download_pubmed_pdf_using_paper_metadata(metadata)
        savepath = f"./document/neurosurgery/{metadata[4]}.pdf"  # 这里用 PMID 存文件
        file = open(savepath, 'wb')
        file.write(pdfstream)
        file.close()
        print('success')
    except Exception as e:
        # 将异常信息写入日志文件
        logging.error(f"Error: {e}, PMID: {metadata[4]}")
    i = i + 1
    if i == length + 1:
        break

1261/2000,1262/2000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7951120/pdf.success
1263/2000,1264/2000,1265/2000,1266/2000,1267/2000,1268/2000,1269/2000,1270/2000,1271/2000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7454774/pdf.success
1272/2000,1273/2000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8782259/pdf.success
1274/2000,1275/2000,1276/2000,1277/2000,1278/2000,1279/2000,1280/2000,1281/2000,1282/2000,1283/2000,1284/2000,1285/2000,1286/2000,1287/2000,1288/2000,1289/2000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2899326/pdf.success
1290/2000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9472586/pdf.success
1291/2000,1292/2000,1293/2000,1294/2000,1295/2000,1296/2000,1297/2000,1298/2000,1299/2000,1300/2000,1301/2000,1302/2000,1303/2000,1304/2000,1305/2000,1306/2000,1307/2000,1308/2000,1309/2000,1310/2000,1311/2000,1312/2000,1313/2000,1314/2000,1315/2000,1316/2000,1317/2000,1318/2000,1319/2000,1320/2000,1321/2000,

IndexError: list index out of range

In [84]:
metadatas = fetch_pdf_urls_from_pubmed(keyword = 'neurosurgery', startpage = 400, endpage = 600)
import logging
from datetime import datetime

# 配置日志
logging.basicConfig(filename='error.log', level=logging.ERROR,
                    format='%(asctime)s:%(levelname)s:%(message)s')


100%|██████████| 200/200 [07:56<00:00,  2.38s/it]


In [85]:
length = len(metadatas)
i = 0
while True:
    print(f'{i}/{length}', end = ',')
    metadata = metadatas[i]
    try:
        pdfstream = download_pubmed_pdf_using_paper_metadata(metadata)
        savepath = f"./document/neurosurgery/{metadata[4]}.pdf"  # 这里用 PMID 存文件
        file = open(savepath, 'wb')
        file.write(pdfstream)
        file.close()
        print('success')
    except Exception as e:
        # 将异常信息写入日志文件
        logging.error(f"Error: {e}, PMID: {metadata[4]}")
    i = i + 1
    if i == length + 1:
        break

0/2000,1/2000,2/2000,3/2000,4/2000,5/2000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2539223/pdf.success
6/2000,7/2000,8/2000,9/2000,10/2000,11/2000,12/2000,13/2000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7414726/pdf.success
14/2000,15/2000,16/2000,17/2000,18/2000,19/2000,20/2000,21/2000,22/2000,23/2000,24/2000,25/2000,26/2000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10016375/pdf.success
27/2000,28/2000,29/2000,30/2000,31/2000,32/2000,33/2000,34/2000,35/2000,36/2000,37/2000,38/2000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10817836/pdf.success
39/2000,40/2000,41/2000,42/2000,43/2000,44/2000,45/2000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8929639/pdf.success
46/2000,47/2000,48/2000,49/2000,50/2000,51/2000,52/2000,53/2000,54/2000,55/2000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9127702/pdf.success
56/2000,57/2000,58/2000,59/2000,60/2000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articl

IndexError: list index out of range

In [86]:
metadatas = fetch_pdf_urls_from_pubmed(keyword = 'neurosurgery', startpage = 600, endpage = 1000)
import logging
from datetime import datetime

# 配置日志
logging.basicConfig(filename='error.log', level=logging.ERROR,
                    format='%(asctime)s:%(levelname)s:%(message)s')


100%|██████████| 400/400 [16:34<00:00,  2.49s/it]


In [87]:
length = len(metadatas)
i = 0
while True:
    print(f'{i}/{length}', end = ',')
    metadata = metadatas[i]
    try:
        pdfstream = download_pubmed_pdf_using_paper_metadata(metadata)
        savepath = f"./document/neurosurgery/{metadata[4]}.pdf"  # 这里用 PMID 存文件
        file = open(savepath, 'wb')
        file.write(pdfstream)
        file.close()
        print('success')
    except Exception as e:
        # 将异常信息写入日志文件
        logging.error(f"Error: {e}, PMID: {metadata[4]}")
    i = i + 1
    if i == length + 1:
        break

0/4000,1/4000,2/4000,3/4000,4/4000,5/4000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5603580/pdf.success
6/4000,7/4000,8/4000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC11070709/pdf.success
9/4000,10/4000,11/4000,12/4000,13/4000,14/4000,15/4000,16/4000,17/4000,18/4000,19/4000,20/4000,21/4000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5030580/pdf.success
22/4000,23/4000,[I] pdf url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8148751/pdf.success
24/4000,25/4000,26/4000,27/4000,28/4000,29/4000,30/4000,31/4000,32/4000,33/4000,34/4000,35/4000,36/4000,37/4000,38/4000,39/4000,40/4000,41/4000,42/4000,43/4000,44/4000,45/4000,46/4000,47/4000,48/4000,49/4000,50/4000,51/4000,52/4000,53/4000,54/4000,55/4000,56/4000,57/4000,58/4000,59/4000,60/4000,61/4000,62/4000,63/4000,64/4000,65/4000,66/4000,67/4000,68/4000,69/4000,70/4000,71/4000,72/4000,73/4000,74/4000,75/4000,76/4000,77/4000,78/4000,79/4000,80/4000,81/4000,82/4000,83/4000,84/4000,85/4000,86/4000,8

IndexError: list index out of range